In [302]:
import pandas as pd
import numpy as np
import json, os
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import glob
import warnings

In [303]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [304]:
import koreanize_matplotlib

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

#pd.Series([1, 3, 5, -7, 9]).plot(title="한글")

In [305]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))

warnings.filterwarnings("ignore")

In [306]:
# 1.0 Training 학습세트에서 100개씩 추출
folders = [
    '01.news_r',
    '02.briefing',
    '03.his_cul',
    '04.paper',
    '05.minute',
    '06.edit',
    '07.public',
    '08.speech',
    '09.literature',
    '10.narration'
    
]

file_limit = 100
file_paths = []

for folder in folders:
    folder_path = f'022.요약문 및 레포트 생성 데이터\\01.데이터\\1.Training\\라벨링데이터\\TL1\\{folder}\\2~3sent\\*.json'
    files = glob.glob(folder_path)[:file_limit]
    file_paths.extend(files)

# Print the resulting file paths
#for file_path in file_paths:
#    print(file_path)
len(file_paths)
#file_paths[:4]

1000

In [307]:
%%time 
# Initialize an empty DataFrame
df = pd.DataFrame(columns=[ "doc_type", "passage"])

# Iterate over each JSON file
for file_path in file_paths:
    # Load the JSON file
    with open(file_path, "r", encoding="utf-8") as file:
        json_data = json.load(file)
    
    # Extract the required information
    doc_type = json_data["Meta(Acqusition)"]["doc_type"]
    passage = json_data["Meta(Refine)"]["passage"]

    # Append a new row to the DataFrame
    #train_data = train_data.append({"doc_type": doc_type, "doc_name": doc_name, "passage_Cnt": passage_Cnt, "passage": passage}, ignore_index=True)
    df.loc[len(df)] = [doc_type, passage]
    
# Print the resulting DataFrame
print(df.shape)
df

(1000, 2)
CPU times: total: 656 ms
Wall time: 689 ms


,doc_type,passage
0,news_r,보수진영 사분오열 속 ‘국민통합연대’ 띄운 비박계 크리스마스를 앞둔 지난 23일 오...
1,news_r,가난 속에서 맨손으로 혼자 창업해 30대 중반에 코스닥 상장까지 일궈낸 이가 있다....
2,news_r,"SK텔레콤은 ‘T끼리 온가족 할인’ 요금제로, 가족 구성원의 가입 합산 기간이..."
3,news_r,“박스 테이프는 어디 갔죠?” 1일 오전 서울 중구의 한 대형마트.\n 장을 본 ...
4,news_r,"현대차그룹 고급 브랜드 제네시스의 첫 스포츠유틸리티차량(SUV), GV80이 드디어..."
...,...,...
995,narration,한국과는 정반대다.\n계절의 뒤바뀜을 느껴보는 것도 여행의 또 다른 재미다.\n이곳...
996,narration,1960년대부터는 중국의 박해를 피해 망명한 티베트인들이 이곳을 중심으로 네팔에 정...
997,narration,포카라에서는 어느 곳에서나 안나푸르나 산 문을 볼 수 있다.\n안나푸르나로 가는 길...
998,narration,청년의 친절이 고마웠다.\n주민들은 산비탈을 깎아 만든 계단식 밭에서 겨울에는 주로...


### 텍스트 전처리

In [308]:
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import nltk

In [309]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\n', ' ', text)
    #text = re.sub('‘', '', text)
    #text = re.sub('’', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub(' +', ' ', text)
    return text

In [310]:
%time df['content_preprocessing'] = df['passage'].apply(preprocessing)

CPU times: total: 15.6 ms
Wall time: 65.5 ms


In [311]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['수', '현', '있는', '있습니다', '그', '년도', '합니다', '하는', 
             '및', '제', '할', '하고', '더', '대한', '한', '그리고', '월', 
             '저는', '없는', '입니다', '등', '일', '많은', '이런', '것은', 
             '왜','같은', '같습니다', '없습니다', '위해', '한다', 
             '여명', '이다', '에', '고', '란', '는', '그래도', '개에', '개인데', '일엔', '있다', '그런데',
            '했다', '그러나', '위한', '것이다', '따라',]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [312]:
%time df['content_preprocessed'] = df['content_preprocessing'].apply(remove_stopwords)

CPU times: total: 31.2 ms
Wall time: 77.6 ms


In [313]:
processed_df = df.drop(columns=['passage', 'content_preprocessing'])
processed_df.head(3)

,doc_type,content_preprocessed
0,news_r,보수진영 사분오열 속 국민통합연대 띄운 비박계 크리스마스를 앞둔 지난 오전 서울 프...
1,news_r,가난 속에서 맨손으로 혼자 창업해 대 중반에 코스닥 상장까지 일궈낸 이가 지난해 상...
2,news_r,SK텔레콤은 T끼리 온가족 할인 요금제로 가족 구성원의 가입 합산 기간이 년을 넘...


In [314]:
processed_df['content_preprocessed'] = processed_df.apply(lambda row: nltk.word_tokenize(row['content_preprocessed']), axis=1)


In [315]:
processed_df.head(3)

,doc_type,content_preprocessed
0,news_r,"[보수진영, 사분오열, 속, 국민통합연대, 띄운, 비박계, 크리스마스를, 앞둔, 지..."
1,news_r,"[가난, 속에서, 맨손으로, 혼자, 창업해, 대, 중반에, 코스닥, 상장까지, 일궈..."
2,news_r,"[SK텔레콤은, T끼리, 온가족, 할인, 요금제로, 가족, 구성원의, 가입, 합산,..."


In [316]:
tokenized_doc = processed_df['content_preprocessed'].apply(lambda x: [word for word in x if len(word) > 1])
print(tokenized_doc[:5])

0    [보수진영, 사분오열, 국민통합연대, 띄운, 비박계, 크리스마스를, 앞둔, 지난, ...
1    [가난, 속에서, 맨손으로, 혼자, 창업해, 중반에, 코스닥, 상장까지, 일궈낸, ...
2    [SK텔레콤은, T끼리, 온가족, 할인, 요금제로, 가족, 구성원의, 가입, 합산,...
3    [박스, 테이프는, 어디, 갔죠, 오전, 서울, 중구의, 대형마트, 장을, 사람들이...
4    [현대차그룹, 고급, 브랜드, 제네시스의, 스포츠유틸리티차량, SUV, GV, 드디...
Name: content_preprocessed, dtype: object


#### 정수 인코딩

In [317]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(41, 1), (42, 1), (101, 1), (115, 1), (154, 1), (212, 3), (213, 1), (214, 1), (215, 1), (216, 1), (217, 1), (218, 1), (219, 1), (220, 1), (221, 1), (222, 1), (223, 1), (224, 1), (225, 1), (226, 1), (227, 1), (228, 1), (229, 1), (230, 1), (231, 1), (232, 1), (233, 1), (234, 1), (235, 1), (236, 1), (237, 1), (238, 1), (239, 1), (240, 1), (241, 1), (242, 1), (243, 4), (244, 1), (245, 1), (246, 1), (247, 1), (248, 1), (249, 1), (250, 1), (251, 1), (252, 1), (253, 1), (254, 1), (255, 1), (256, 1), (257, 1), (258, 2), (259, 3), (260, 1), (261, 1), (262, 1), (263, 1), (264, 1), (265, 1), (266, 1), (267, 1), (268, 1), (269, 1), (270, 1), (271, 1), (272, 1), (273, 1), (274, 1), (275, 1), (276, 1), (277, 1), (278, 1), (279, 2), (280, 1), (281, 1), (282, 1), (283, 1), (284, 1), (285, 1), (286, 1), (287, 1), (288, 1), (289, 1), (290, 1), (291, 1), (292, 1), (293, 1), (294, 1), (295, 1), (296, 1), (297, 2), (298, 1), (299, 3), (300, 1), (301, 1), (302, 1), (303, 1), (304, 1), (305, 1), (306, 1), (

In [318]:
print(dictionary[425])

회사


In [319]:
len(dictionary)
#총 62019 개의 단어가 학습되었다

62019

### LDA 모델 훈련
10개의 카테고리를 가지고 있으므로 토픽의 개수를 10으로 하여 학습

In [322]:
import gensim
NUM_TOPICS = 10 # 10 토픽, k=10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4) #각 토픽의 상위 단어를 가져온다. 여기서는 상위 4개 단어 출력 
for topic in topics:
    print(topic)

(0, '0.002*"그는" + 0.002*"때문에" + 0.001*"함께" + 0.001*"것으로"')
(1, '0.003*"위원" + 0.002*"그런" + 0.002*"지금" + 0.002*"것을"')
(2, '0.005*"위원" + 0.002*"것이" + 0.002*"그런" + 0.002*"이렇게"')
(3, '0.002*"것으로" + 0.001*"코로나" + 0.001*"통해" + 0.001*"대해"')
(4, '0.002*"것으로" + 0.002*"때문에" + 0.001*"경우" + 0.001*"없다"')
(5, '0.002*"것이" + 0.002*"것으로" + 0.002*"때문에" + 0.001*"위원"')
(6, '0.002*"지금" + 0.002*"위원" + 0.001*"것을" + 0.001*"이에"')
(7, '0.002*"함께" + 0.001*"때문에" + 0.001*"것으로" + 0.001*"그는"')
(8, '0.001*"것이" + 0.001*"것으로" + 0.001*"또는" + 0.001*"다시"')
(9, '0.001*"것을" + 0.001*"것이" + 0.001*"때문에" + 0.001*"그는"')


각 단어 앞에 붙은 수피가 단어의 해당 토픽에 대한 기여도.

In [320]:
print(ldamodel.print_topics())


[(0, '0.002*"것으로" + 0.002*"경우" + 0.002*"함께" + 0.001*"때문에" + 0.001*"하지만" + 0.001*"것을" + 0.001*"현재" + 0.001*"후보가" + 0.001*"것이" + 0.001*"그런"'), (1, '0.003*"위원" + 0.002*"것으로" + 0.001*"것이" + 0.001*"지금" + 0.001*"때문에" + 0.001*"것을" + 0.001*"등을" + 0.001*"지난해" + 0.001*"어떤" + 0.001*"그런"'), (2, '0.002*"것으로" + 0.002*"그런" + 0.002*"지금" + 0.001*"위원" + 0.001*"때문에" + 0.001*"많이" + 0.001*"경우" + 0.001*"것을" + 0.001*"것이" + 0.001*"그래서"'), (3, '0.001*"것이" + 0.001*"그런" + 0.001*"것으로" + 0.001*"때문에" + 0.001*"지금" + 0.001*"통해" + 0.001*"이렇게" + 0.001*"모든" + 0.001*"이러한" + 0.001*"아니라"'), (4, '0.002*"위원" + 0.002*"때문에" + 0.001*"것으로" + 0.001*"것을" + 0.001*"지금" + 0.001*"것이" + 0.001*"그런" + 0.001*"하여" + 0.001*"당시" + 0.001*"지원"'), (5, '0.003*"위원" + 0.002*"지금" + 0.002*"그는" + 0.002*"이렇게" + 0.001*"그런" + 0.001*"그렇게" + 0.001*"때문에" + 0.001*"서남수" + 0.001*"것이" + 0.001*"교육부장관"'), (6, '0.001*"있었다" + 0.001*"등을" + 0.001*"것으로" + 0.001*"것이" + 0.001*"것을" + 0.001*"가장" + 0.001*"때문에" + 0.001*"후보가" + 0.001*"특히" + 0.001*"우리"'), (7, '0.002*"것이" + 0

#### LDA 시각화

In [333]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)


### 문서별 토픽 분포

In [ ]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)


### TF-IDF 행렬 만들기

In [326]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(processed_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 다시 text['headline_text']에 재저장
processed_df['content_preprocessed'] = detokenized_doc


In [327]:
processed_df['content_preprocessed'][:5]

0    보수진영 사분오열 국민통합연대 띄운 비박계 크리스마스를 앞둔 지난 오전 서울 프레스...
1    가난 속에서 맨손으로 혼자 창업해 중반에 코스닥 상장까지 일궈낸 이가 지난해 상장한...
2    SK텔레콤은 T끼리 온가족 할인 요금제로 가족 구성원의 가입 합산 기간이 년을 넘으...
3    박스 테이프는 어디 갔죠 오전 서울 중구의 대형마트 장을 사람들이 자율포장대 위에서...
4    현대차그룹 고급 브랜드 제네시스의 스포츠유틸리티차량 SUV GV 드디어 모습을 드러...
Name: content_preprocessed, dtype: object

In [328]:
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words=["돋움", "경우", "또는", "있습니다", "있는", "합니다"], max_features= 1000)
X = vectorizer.fit_transform(processed_df['content_preprocessed'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)


TF-IDF 행렬의 크기 : (1000, 1000)


### 토픽 모델링

In [329]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)


In [330]:
lda_top = lda_model.fit_transform(X)


In [331]:
print(lda_model.components_)
print(lda_model.components_.shape) 


[[1.22076316 0.30952363 1.24319167 ... 1.5942066  3.44103511 1.98326271]
 [0.6966638  0.61074252 1.42432695 ... 1.1746519  0.33059997 0.36762952]
 [0.31673754 0.39224793 0.32268267 ... 0.40960997 0.37030482 0.61766828]
 ...
 [0.36328516 0.33014506 0.33015343 ... 0.34855395 0.34017622 0.59898284]
 [0.33019604 0.31254777 0.49081837 ... 0.34747798 0.44910239 0.35614617]
 [0.3116456  1.14065319 0.44757084 ... 0.31395593 0.55037824 0.35581445]]
(10, 1000)


In [332]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)


Topic 1: [('것이', 13.68), ('나는', 11.8), ('때문에', 11.75), ('것을', 11.46), ('가장', 10.84)]
Topic 2: [('민주당', 2.82), ('대통령', 2.31), ('윤석열', 2.31), ('의원', 2.25), ('후보', 2.24)]
Topic 3: [('후보가', 1.19), ('문재인', 1.16), ('정권의', 1.1), ('것으로', 1.01), ('자동차', 0.99)]
Topic 4: [('위원', 3.32), ('지금', 2.53), ('그런', 2.29), ('무슨', 1.9), ('그래서', 1.84)]
Topic 5: [('후보는', 3.62), ('후보가', 2.79), ('후보', 2.0), ('이인제', 1.82), ('후보의', 1.75)]
Topic 6: [('곳이다', 4.1), ('조선', 2.64), ('있으며', 2.61), ('만든', 2.46), ('년에', 2.41)]
Topic 7: [('위원', 2.02), ('상식의', 1.79), ('김상희', 1.34), ('행위', 1.2), ('지금', 1.18)]
Topic 8: [('위원', 3.44), ('그런', 2.43), ('서남수', 2.21), ('말씀을', 2.14), ('이렇게', 2.09)]
Topic 9: [('만원', 1.28), ('억원', 1.07), ('억원을', 1.05), ('위원', 1.02), ('사업의', 0.95)]
Topic 10: [('것으로', 1.49), ('세운', 1.39), ('위원', 1.29), ('지금', 1.22), ('지원', 1.22)]
